In [1]:
import sys
sys.path.append("/Users/jrand/git-repos/AIE7/11_Certification_Challenge/backend/src")

In [8]:
from mimi.pipe.populate_all import main

main()

🚀 Starting population for all chunker types...
Found 2 chunker types: ['naive_chunker', 'markdown_chunker']

🔄 Populating for chunker: naive_chunker
Initializing config with: {'agent_type': 'supervisor', 'chunker_type': 'naive_chunker', 'retriever_type': 'cosine'}
Config initialized successfully: {'agent_type': 'supervisor', 'chunker_type': 'naive_chunker', 'retriever_type': 'cosine'}
Starting pipeline with docs path: data
Searching for .md files in: /Users/jrand/git-repos/AIE7/11_Certification_Challenge/notebooks/data
Found 0 .md files: []
Total documents loaded: 0
Documents loaded: 0
Using chunker: naive_chunker
Chunks created: 0
❌ Failed to populate for naive_chunker: list index out of range
Full traceback:
Resetting config for naive_chunker
❌ Failed to populate for naive_chunker: list index out of range
Full traceback:

🔄 Populating for chunker: markdown_chunker
Initializing config with: {'agent_type': 'supervisor', 'chunker_type': 'markdown_chunker', 'retriever_type': 'cosine'}
Co

Traceback (most recent call last):
  File "/Users/jrand/git-repos/AIE7/11_Certification_Challenge/backend/src/mimi/pipe/populate_all.py", line 40, in populate_for_chunker
    result = pipe(docs_path)
  File "/Users/jrand/git-repos/AIE7/11_Certification_Challenge/backend/src/mimi/pipe/pipe.py", line 28, in pipe
    vectorstore = index(chunks)
  File "/Users/jrand/git-repos/AIE7/11_Certification_Challenge/backend/src/mimi/pipe/index.py", line 14, in index
    qdrant_vectorstore = Qdrant.from_documents(
        documents=chunks,
    ...<3 lines>...
        force_recreate=True  # Overwrite existing collection
    )
  File "/Users/jrand/git-repos/AIE7/11_Certification_Challenge/.venv/lib/python3.13/site-packages/langchain_core/vectorstores/base.py", line 848, in from_documents
    return cls.from_texts(texts, embedding, metadatas=metadatas, **kwargs)
           ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jrand/git-repos/AIE7/11_Certification_Challenge/.ven

In [2]:
from mimi.evals.ragas_eval import generate_all_invocables
from mimi.evals.ragas_csv_loader import load_ragas_csv
from mimi.evals.ragas_eval import run_ragas_eval_parallel, run_ragas_eval

/Users/jrand/git-repos/AIE7/11_Certification_Challenge/backend/src/mimi/agents/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from mimi.agents.supervisor import create_supervisor_graph


In [3]:
# Generate all combinations
invocables = generate_all_invocables()

In [4]:
# Load dataset
dataset = load_ragas_csv(f"../data/sdg/testset.csv")

In [5]:
# Run all experiments - Qdrant won't let them run in parallel
# results = run_ragas_eval_parallel(invocables, dataset)

In [6]:
results_d = {
    invocable_name: run_ragas_eval(invocable, invocable_name, dataset)
    for invocable_name, invocable 
    in invocables.items()
}

ValueError: Collection naive_chunker not found

In [55]:
import pandas as pd

score_dfs = []
for name, result in results_d.items():
    score_df = pd.DataFrame(result.scores)
    score_df['experiment_name'] = name
    score_dfs.append(score_df)

score_df = pd.concat(score_dfs)
score_df

,context_recall,faithfulness,factual_correctness,answer_relevancy,experiment_name
0,1.0,0.555556,0.18,0.000000,naive
1,1.0,0.522727,0.15,0.941829,naive
2,1.0,0.666667,0.50,0.987586,naive
3,0.0,0.000000,0.24,0.960614,naive
4,1.0,0.941176,0.71,0.944441,naive
...,...,...,...,...,...
5,1.0,0.878788,0.62,0.000000,semantic
6,NaN,0.880952,0.88,0.951291,semantic
7,1.0,0.750000,0.37,0.000000,semantic
8,1.0,0.825000,NaN,0.934657,semantic


In [ ]:
import pandas as pd

score_df = pd.read_csv("../data/experiments/score_df_123e4567-e89b-12d3-a456-426614174000.csv")

score_df.head()

In [56]:
score_aggs = score_df.groupby('experiment_name').mean()
score_aggs

,context_recall,faithfulness,factual_correctness,answer_relevancy
experiment_name,,,,
bm25,0.633333,0.640254,0.562222,0.749486
contextual_compression,0.683333,0.768561,0.612857,0.842251
ensemble,0.966667,0.858049,0.455000,0.665007
multi_query,1.000000,0.900093,0.616667,0.735416
naive,0.833333,0.731857,0.507500,0.824444
parent_document,0.833333,0.787431,0.475000,0.742986
semantic,1.000000,0.777018,0.578571,0.663101


In [59]:
tags

dict_keys(['naive', 'bm25', 'contextual_compression', 'multi_query', 'parent_document', 'ensemble', 'semantic'])

In [62]:
from collections import defaultdict
from langsmith import Client

client = Client()

tags = list(invocables.keys())
by_tag = defaultdict(dict)

for t in tqdm(tags):
    by_tag[t] = client.get_run_stats(
        project_names=[os.environ["LANGSMITH_PROJECT"]],
        # filter syntax lets you match an element in the tags array
        filter=f"has(tags, '{t}')"
    )

In [67]:
langsmith_df = pd.DataFrame(by_tag).T
langsmith_df

,run_count,latency_p50,latency_p99,first_token_p50,first_token_p99,total_tokens,prompt_tokens,completion_tokens,median_tokens,completion_tokens_p50,...,last_run_start_time,feedback_stats,run_facets,error_rate,streaming_rate,total_cost,prompt_cost,completion_cost,cost_p50,cost_p99
naive,140,0.097,5.97622,None,None,78448,75422,3026,0,0,...,2025-07-29T18:07:50.680775,{},"[{'key': 'name', 'value': 'RunnableLambda', 'q...",0.0,0.0,0.008753,0.007542,0.00121,0.000856,0.00128
bm25,140,0.004,5.53122,None,None,44496,41779,2717,0,0,...,2025-07-29T18:07:44.704082,{},"[{'key': 'name', 'value': 'RunnableLambda', 'q...",0.0,0.0,0.005265,0.004178,0.001087,0.000536,0.000795
contextual_compression,150,0.2925,8.65653,None,None,29567,27045,2522,0,0,...,2025-07-29T18:07:49.042365,{},"[{'key': 'name', 'value': 'RunnableLambda', 'q...",0.0,0.0,0.003713,0.002704,0.001009,0.000342,0.000643
multi_query,210,0.267,10.06614,None,None,109729,105427,4302,0,0,...,2025-07-29T18:08:26.420350,{},"[{'key': 'name', 'value': 'RunnableLambda', 'q...",0.0,0.0,0.01163,0.009909,0.001721,0.000288,0.001536
parent_document,140,0.111,11.36261,None,None,43181,40126,3055,0,0,...,2025-07-29T18:08:05.123571,{},"[{'key': 'name', 'value': 'RunnableLambda', 'q...",0.0,0.0,0.005235,0.004013,0.001222,0.000406,0.000912
ensemble,270,0.311,24.77662,None,None,161680,157113,4567,0,0,...,2025-07-29T18:08:55.882714,{},"[{'key': 'name', 'value': 'VectorStoreRetrieve...",0.0,0.0,0.017538,0.015711,0.001827,0.000487,0.002717
semantic,140,0.089,9.86083,None,None,67636,64023,3613,0,0,...,2025-07-29T18:07:50.507913,{},"[{'key': 'name', 'value': 'RunnableLambda', 'q...",0.0,0.0,0.007848,0.006402,0.001445,0.000821,0.000982


In [71]:
score_aggs_combined = pd.concat([score_aggs, -langsmith_df[['latency_p50', 'latency_p99', 'cost_p50', 'cost_p99']]], axis=1)
score_aggs_combined

,context_recall,faithfulness,factual_correctness,answer_relevancy,latency_p50,latency_p99,cost_p50,cost_p99
bm25,0.633333,0.640254,0.562222,0.749486,-0.004,-5.53122,-0.000536,-0.000795
contextual_compression,0.683333,0.768561,0.612857,0.842251,-0.2925,-8.65653,-0.000342,-0.000643
ensemble,0.966667,0.858049,0.455000,0.665007,-0.311,-24.77662,-0.000487,-0.002717
multi_query,1.000000,0.900093,0.616667,0.735416,-0.267,-10.06614,-0.000288,-0.001536
naive,0.833333,0.731857,0.507500,0.824444,-0.097,-5.97622,-0.000856,-0.00128
parent_document,0.833333,0.787431,0.475000,0.742986,-0.111,-11.36261,-0.000406,-0.000912
semantic,1.000000,0.777018,0.578571,0.663101,-0.089,-9.86083,-0.000821,-0.000982


In [75]:
score_aggs_combined.rank(ascending=False)

,context_recall,faithfulness,factual_correctness,answer_relevancy,latency_p50,latency_p99,cost_p50,cost_p99
bm25,7.0,7.0,4.0,3.0,1.0,1.0,5.0,2.0
contextual_compression,6.0,5.0,2.0,1.0,6.0,3.0,2.0,1.0
ensemble,3.0,2.0,7.0,6.0,7.0,7.0,4.0,7.0
multi_query,1.5,1.0,1.0,5.0,5.0,5.0,1.0,6.0
naive,4.5,6.0,5.0,2.0,3.0,2.0,7.0,5.0
parent_document,4.5,3.0,6.0,4.0,4.0,6.0,3.0,3.0
semantic,1.5,4.0,3.0,7.0,2.0,4.0,6.0,4.0


In [ ]:
# aggregated rankings for evals only
score_aggs.rank(ascending=False).mean(axis=1).sort_values(ascending=True)

experiment_name
multi_query               2.125
contextual_compression    3.500
semantic                  3.875
naive                     4.375
parent_document           4.375
ensemble                  4.500
bm25                      5.250
dtype: float64

In [77]:
# aggregated rankings for cost/latency only
(-langsmith_df[['latency_p50', 'latency_p99', 'cost_p50', 'cost_p99']]).rank(ascending=False).mean(axis=1).sort_values(ascending=True)

bm25                      2.25
contextual_compression    3.00
parent_document           4.00
semantic                  4.00
naive                     4.25
multi_query               4.25
ensemble                  6.25
dtype: float64

In [ ]:
# aggregated rankings for both evals and cost/latency
score_aggs_combined.rank(ascending=False).mean(axis=1).sort_values(ascending=True)

multi_query               3.1875
contextual_compression    3.2500
bm25                      3.7500
semantic                  3.9375
parent_document           4.1875
naive                     4.3125
ensemble                  5.3750
dtype: float64

# Conclusion

The winner of my experiments is `multi_query`, which provides the optimal balance of quality, latency, and cost. The cell below provides a complete picture of how each retriever ranks compared to all other retrievers on 8 different dimensions.

The rankings largely meet expectations. For example, bm25 is fast but has the worst accuracy. The ensemble is slow and expensive, but did not produce enough accuracy boost to justify.

In [78]:
score_aggs_combined.rank(ascending=False)

,context_recall,faithfulness,factual_correctness,answer_relevancy,latency_p50,latency_p99,cost_p50,cost_p99
bm25,7.0,7.0,4.0,3.0,1.0,1.0,5.0,2.0
contextual_compression,6.0,5.0,2.0,1.0,6.0,3.0,2.0,1.0
ensemble,3.0,2.0,7.0,6.0,7.0,7.0,4.0,7.0
multi_query,1.5,1.0,1.0,5.0,5.0,5.0,1.0,6.0
naive,4.5,6.0,5.0,2.0,3.0,2.0,7.0,5.0
parent_document,4.5,3.0,6.0,4.0,4.0,6.0,3.0,3.0
semantic,1.5,4.0,3.0,7.0,2.0,4.0,6.0,4.0
